In [1]:

# Import pandas library
import pandas as pd

In [2]:
# initialize list
df = pd.read_csv('M&M Department.csv')
 
# initialize list of lists
df2 = pd.read_csv('M&M Title.csv')
 
# initialize list of lists
df3 = pd.read_csv('M&M payroll Name.csv')

Essentially, we need to cross join DF1, DF2, DF3

In [3]:
# Now to perform cross join, we will create
# a key column in both the DataFrames to 
# merge on that key.
df['key'] = 1
df2['key'] = 1
df3['key'] = 1

In [4]:
# to obtain the cross join we will merge on 
# the key - need to keep the key so we can merge with DF3
result = pd.merge(df, df2, on ='key')
result

,Department,key,Title
0,Accounting,1,Attorney
1,Accounting,1,Litigation Paralegal
2,Accounting,1,Case Manager
3,Accounting,1,Legal Assistant
4,Accounting,1,Pre-Suit Secretary
...,...,...,...
240,Worker's Compensation,1,Attorney
241,Worker's Compensation,1,Litigation Paralegal
242,Worker's Compensation,1,Case Manager
243,Worker's Compensation,1,Legal Assistant


In [13]:
# Last merge result with DF3 on the key and now we drop it.
result2 = pd.merge(result, df3, on ='key').drop("key", 1)
  
result2.head(20)

/var/folders/zr/p3dmj0t57nq_9sd5gmyrfgy40000gq/T/ipykernel_41496/1847778353.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  result2 = pd.merge(result, df3, on ='key').drop("key", 1)


,Department,Title,Payroll Name
0,Accounting,Attorney,Alabama
1,Accounting,Attorney,Arizona
2,Accounting,Attorney,Atlanta
3,Accounting,Attorney,Birmingham
4,Accounting,Attorney,Boston
5,Accounting,Attorney,Bowling Green
6,Accounting,Attorney,Brooklyn
7,Accounting,Attorney,Case Control Center
8,Accounting,Attorney,Columbus
9,Accounting,Attorney,Complex Lit


In [14]:
result2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6615 entries, 0 to 6614
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Department    6615 non-null   object
 1   Title         6615 non-null   object
 2   Payroll Name  6615 non-null   object
dtypes: object(3)
memory usage: 206.7+ KB


In [15]:
#add color column - we don't want color column to be cross joined with the other values - just added as a column at the end. 
result2['color'] = '#ffc940'
result2['options'] = '{' + '}'
result2['label'] = result2['Department'] + '-' + result2['Title'] + '-' + result2['Payroll Name']
result2.head()

,Department,Title,Payroll Name,color,options,label
0,Accounting,Attorney,Alabama,#ffc940,{},Accounting-Attorney-Alabama
1,Accounting,Attorney,Arizona,#ffc940,{},Accounting-Attorney-Arizona
2,Accounting,Attorney,Atlanta,#ffc940,{},Accounting-Attorney-Atlanta
3,Accounting,Attorney,Birmingham,#ffc940,{},Accounting-Attorney-Birmingham
4,Accounting,Attorney,Boston,#ffc940,{},Accounting-Attorney-Boston


Now we create an `y` column with our CQL expression in it

In [19]:
# "y": "sum{change:depart if department=\"	\" && title=\"	\" && payrollName=\" 	\"

#concat `CQL` expression:
result2['y'] = '\"sum{change:depart if department=\"' + result2['Department'] + '\" && title=\"' + result2['Title'] + '\" && payrollName=\"' + result2['Payroll Name'] + '\"' + "}"

#change data type in this whole column as string:
result2['y'].astype('string')

#visual check to make sure it works
result2.head()

,Department,Title,Payroll Name,color,options,label,y
0,Accounting,Attorney,Alabama,#ffc940,{},Accounting-Attorney-Alabama,"""sum{change:depart if department=""Accounting"" ..."
1,Accounting,Attorney,Arizona,#ffc940,{},Accounting-Attorney-Arizona,"""sum{change:depart if department=""Accounting"" ..."
2,Accounting,Attorney,Atlanta,#ffc940,{},Accounting-Attorney-Atlanta,"""sum{change:depart if department=""Accounting"" ..."
3,Accounting,Attorney,Birmingham,#ffc940,{},Accounting-Attorney-Birmingham,"""sum{change:depart if department=""Accounting"" ..."
4,Accounting,Attorney,Boston,#ffc940,{},Accounting-Attorney-Boston,"""sum{change:depart if department=""Accounting"" ..."


In [20]:
result2['y'][4]

'"sum{change:depart if department="Accounting" && title="Attorney" && payrollName="Boston"}'

In [21]:
tojson = result2.loc[:, ['label', 'color', 'y', 'options']]

In [22]:
#send it to JSON Town! 
tojson.to_json('result2.json', orient='records')